In [1]:
from models_and_tools.ACGAN_paper import *
from models_and_tools.ACGAN_simple import *
from models_and_tools.functions import *
from skimage.metrics import structural_similarity as ssim
device = 'cuda'

Downloading the model and parameters

In [2]:
netG = Generator(conf).to(device)
netD = Discriminator(conf).to(device)
# Load the weights back into the models
netG.load_state_dict(torch.load('netG200.pth'))
netD.load_state_dict(torch.load('netD200.pth'))
# Make sure to call eval() if you're in inference mode
netG.eval()
netD.eval()

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Dropout(p=0.5, inplace=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.2, inplace=True)
    (10): Dropout(p=0.5, inplace=False)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.2, inplace=True)
    (14): Dropout(p=0.5, inplace=False)
    (15): Conv2d(128, 

Defining functions required for the SSIM score calculations

In [3]:
def rgb_to_grayscale_numpy(image):
	"""
	Convert an RGB image to a grayscale image using the luminosity method.
	Parameters:
		image (numpy.ndarray): The RGB image in CHW format.
	Returns:
		numpy.ndarray: The grayscale image in 1HW format (1 channel, Height, Width).
	"""
	if image.shape[0] == 3:  # Check if the input image has 3 channels (RGB)
		image = image.transpose(1, 2, 0)  # Convert to HWC format for easier processing
	# Apply the luminosity method to calculate grayscale values
	grayscale = 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]
	return grayscale[np.newaxis, :, :]  # Add channel dimension back for consistency


def calculate_ssim(image1, image2):
	"""
    Calculate the Structural Similarity Index Measure (SSIM) between two images.
    Parameters:
        image1, image2 (numpy.ndarray or torch.Tensor): The input images.
    Returns:
        float: The SSIM index between the two input images.
    """
	# Convert PyTorch tensors to numpy arrays if necessary
	if torch.is_tensor(image1):
		image1 = image1.cpu().numpy()  # CHW to HWC
	if torch.is_tensor(image2):
		image2 = image2.cpu().numpy()  # CHW to HWC
	# Convert images to grayscale
	image1 = rgb_to_grayscale_numpy(image1)[0]
	image2 = rgb_to_grayscale_numpy(image2)[0]
	# Normalize images to ensure they are compared on the same scale
	image1 = (image1 - image1.min()) / (image1.max() - image1.min())
	image2 = (image2 - image2.min()) / (image2.max() - image2.min())
	# Calculate and return the SSIM
	ssim_value = ssim(image1, image2, multichannel=False, data_range=1)
	return ssim_value

Here we need to generate 2 times 1000 images using our GAN model named fake_images1 and fake_images2. For AC-GAN it's a little bit different compared to  normal GANS.

In [4]:
num_images_acc = 1000  # number of images
noise = torch.randn(num_images_acc, 100, device=device)  #noise for images
# creating labels (required only for AC-GAM)
fake_labels = torch.randint(0, conf.num_classes, (num_images_acc,), device=device)
fake_labels_oh = torch.nn.functional.one_hot(fake_labels, num_classes=10)  # one-hot representation
# generating images - 1st pack
fake_images = netG(noise, fake_labels_oh).detach()
# Now repeating the same procedure for the 2nd pack of the images
noise2 = torch.randn(num_images_acc, 100, device=device)
fake_labels2 = fake_labels
fake_labels_oh2 = torch.nn.functional.one_hot(fake_labels, num_classes=10)
# generating images - 1st pack
fake_images2 = netG(noise2, fake_labels_oh2).detach()
# Calculate SSIM for each pair


Calculating the average SSIM score.

In [5]:
ssim_scores = []
for gen_img, gen_img2 in zip(fake_images, fake_images2):  # 2 packs pairwise
	current_ssim = calculate_ssim(gen_img, gen_img2)
	ssim_scores.append(current_ssim)

average_ssim_ACGAN = np.mean(ssim_scores)
print(f"Average SSIM (AC-GAN): {average_ssim_ACGAN}")

Average SSIM (AC-GAN): 0.08215493456964973


This number will be placed in 1 plot, comparing all the models